In [ ]:
import torch
import numpy as np
from configuration import config_jup
from utils.data_loader import get_loader_all_clients
from utils.train_utils import get_logger, initialize_clients, FedAvg, weightedFedAvg, test_global_model, compute_avg_acc_for, save_results
from datetime import datetime

c:\Users\VARUN\OneDrive\Documents\Semester_7\RepresentationLearning\online-FCL-Improve\ofcl\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
args = config_jup.base_parser() # load the default arguments
if torch.cuda.is_available():
    args.cuda = True
    args.device = f'cuda:0'
else:
    args.device = 'cpu' 

usage: ipykernel_launcher.py [-h] [--framework FRAMEWORK]
                             [--dir_data DIR_DATA] [--dir_output DIR_OUTPUT]
                             [--dataset_name DATASET_NAME]
                             [--model_name MODEL_NAME]
                             [--batch_size BATCH_SIZE] [--lr LR]
                             [--optimizer OPTIMIZER] [--temp TEMP]
                             [--sup_type SUP_TYPE]
                             [--local_epochs LOCAL_EPOCHS]
                             [--non_augment_epochs NON_AUGMENT_EPOCHS]
                             [--n_runs N_RUNS] [--n_tasks N_TASKS]
                             [--with_memory WITH_MEMORY]
                             [--memory_size MEMORY_SIZE]
                             [--update_strategy UPDATE_STRATEGY]
                             [--sampling_strategy SAMPLING_STRATEGY]
                             [--balanced_update BALANCED_UPDATE]
                             [--uncertainty_score UNCERTAI

SystemExit: 2

c:\Users\VARUN\OneDrive\Documents\Semester_7\RepresentationLearning\online-FCL-Improve\ofcl\Lib\site-packages\IPython\core\interactiveshell.py:3707: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


: 

In [3]:
print(args.device)

cuda:0


In [6]:
# change the default arguments if needed (see example below)
args.dataset_name = 'cifar10'
args.memory_size = 200
logger = get_logger(args)
args.n_runs = 1
print(args)

Namespace(framework='FCL', dir_data='./data/', dir_output='./output/', dataset_name='cifar10', model_name='resnet', batch_size=10, lr=0.1, optimizer='sgd', local_epochs=3, n_runs=1, n_tasks=5, with_memory=1, memory_size=200, update_strategy='balanced', sampling_strategy='random', balanced_update='uncertainty', uncertainty_score='bregman', subsample_size=50, balanced_step='bottomk', n_clients=5, overlap='overlap', burnin=30, jump=5, fl_update='w_favg', mu=0.01, cuda=True, device='cuda:0', input_size=(3, 32, 32), n_classes=10, n_classes_per_task=2, dir_results='./output//FCL/cifar10/w_favg/overlap/5clients/5tasks/30/5/resnet/sgd/01/200/10/3/random/balanced_uncertainty/bregman/bottomk/')


In [5]:
for run in range(args.n_runs):
    loader_clients, cls_assignment_list, global_test_loader = get_loader_all_clients(args, run)
    clients = initialize_clients(args, loader_clients, cls_assignment_list, run)

    start_time = datetime.now()
    for client in clients:
        print(client.model)
    
    # while not all([client.train_completed for client in clients]):
    #     for client in clients:
    #         if not client.train_completed:
    #             samples, labels = client.get_next_batch()
    #             if samples is not None:
    #                 if args.with_memory:
    #                     if client.task_id == 0:
    #                         client.train_with_update(samples, labels)
    #                     else:
    #                         client.train_with_memory(samples, labels)
    #                 else:
    #                     client.train(samples, labels)
    #             else:
    #                 print(f'Run {run} - Client {client.client_id} - Task {client.task_id} completed - {client.get_current_task()}')
    #                 # compute loss train
    #                 logger = client.compute_loss(logger, run)
    #                 print(f'Run {run} - Client {client.client_id} - Test time - Task {client.task_id}')
    #                 logger = client.test(logger, run)
    #                 logger = client.validation(logger, run)
    #                 logger = client.forgetting(logger, run)

    #                 if client.task_id + 1 >= args.n_tasks:
    #                     client.train_completed = True
    #                     print(f'Run {run} - Client {client.client_id} - Train completed')
    #                     logger = client.balanced_accuracy(logger, run)
    #                 else:
    #                     client.task_id += 1

    #     # COMMUNICATION ROUND PART
    #     selected_clients = [client.client_id for client in clients if (client.num_batches >= args.burnin and client.num_batches % args.jump == 0 and client.train_completed == False)]
    #     if len(selected_clients) > 1:
    #         # communication round when all clients process a mini-batch
    #         if args.fl_update.startswith('w_'):
    #             global_model = weightedFedAvg(args, selected_clients, clients)
    #         else:
    #             global_model = FedAvg(args, selected_clients, clients)

    #         global_parameters = global_model.state_dict()
    #         # local models update with averaged global parameters
    #         for client_id in selected_clients:
    #             clients[client_id].save_last_local_model()
    #             clients[client_id].update_parameters(global_parameters)
    #             clients[client_id].save_last_global_model(global_model)

    # end_time = datetime.now()
    # print(f'Duration: {end_time - start_time}')
    # # global model accuracy when all clients finish their training on all tasks (FedCIL ICLR2023)
    # logger = test_global_model(args, global_test_loader, global_model, logger, run)

./data//data_splits/FCL/cifar10/overlap/5clients/5tasks/run0/cifar10_split.pkl


AttributeError: 'Namespace' object has no attribute 'temp'

In [ ]:
for client_id in range(args.n_clients):
    print(f'Client {client_id}: {clients[client_id].task_list}')
    print(np.mean(logger['test']['acc'][client_id], 0))
    final_acc = np.mean(np.mean(logger["test"]["acc"][client_id], 0)[args.n_tasks-1,:], 0)
    final_for = np.mean(logger["test"]["forget"][client_id])
    final_bal_acc = np.mean(logger["test"]["bal_acc"][client_id])
    print(f'Final client accuracy: {final_acc:0.4f}')
    print(f'Final client forgetting: {final_for:0.4f}')
    print(f'Final client balanced accuracy: {final_bal_acc:0.4f}')
    print()

In [ ]:
mean_acc, std_acc, mean_for, std_for = compute_avg_acc_for(args, logger)
print(f'Final average accuracy: {mean_acc:0.4f} (+-) {std_acc:0.4f}')
print(f'Final average forgetting: {mean_for:0.4f} (+-) {std_for:0.4f}')
print()

In [ ]:
# save training results
save_results(args, logger)